In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
import scipy as sc
yf.pdr_override()

In [11]:
#import data
def getsp_list():
    # Get list of S&P stocks from wiki
    sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    data_table = pd.read_html(sp500url)
    # Place list of stocks in a table
    snp_list = data_table[0]
    # remove unneeded columns
    snp_list = snp_list.drop(columns=["CIK","Founded","Headquarters Location"], inplace=False)
    # set index
    snp_list = snp_list.set_index("Symbol",inplace=False)
    # set convert add dates to datetime format
    snp_list['Date added'] = pd.to_datetime(snp_list['Date added'], errors='coerce')
    full_list = snp_list.index.T.to_list()
    return full_list

def getData(stocks, start, end):
    stockdata = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockdata = stockdata['Adj Close']
    
    returns = stockdata.pct_change().dropna(how='all').dropna(axis=1)
    return_list = returns.columns.T.to_list()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix, return_list

# Calculate portfolio performance
def portfolio_perform(weights, mean_returns, cov_matrix):  
    returns = np.sum(mean_returns*weights)*252
    std = np.sqrt( np.dot(pd.DataFrame(weights).T.values.tolist(), np.dot(cov_matrix, weights)) ) * np.sqrt(252)
    return returns, std

def port_std(weights, cov_matrix):
    std = np.sqrt( np.dot(pd.DataFrame(weights).T.values.tolist(), np.dot(cov_matrix, weights)) ) * np.sqrt(252)
    return std

def neg_sharperatio(weights, mean_returns, cov_matrix, riskfreerate = .03):
    preturns, pstd = portfolio_perform(weights, mean_returns, cov_matrix)
    return - (preturns- riskfreerate)/pstd
    
def max_sf(mean_returns, cov_matrix, riskfreerate = .03, constraint_set=(0,.05)):
    # Minimizing the negative SR by alering the weights of the portfolio
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, riskfreerate)
    # Use no risk constraint or constraint set up by user
    if 1==1:
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    else:
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                       {'type': 'eq', 'fun': lambda x: (np.sqrt( np.dot(pd.DataFrame(x).T.values.tolist(), np.dot(cov_matrix, x)) ) * np.sqrt(252))-.25})

    bound = constraint_set
    bounds = tuple(bound for asset in range(num_assets))
    result = sc.optimize.minimize(neg_sharperatio, num_assets*[1/num_assets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [9]:
stocklist = ['CBA','TSLA','AAPL', 'GOOG', 'SPY', 'F', 'LQDI', 'BNDX']
weights = [.03,.03,.04]

end1 = dt.date.today()
start1 = end1 - dt.timedelta(days=21*365)
start = start1.strftime('%Y-%m-%d')
end = end1.strftime('%Y-%m-%d')

In [4]:
full_list = getsp_list()
mean_returns, cov_matrix, return_list = getData(stocks=full_list,start=start,end=end)
# returns, std = portfolio_perform(weights, mean_returns, cov_matrix)

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted


FileNotFoundError: [Errno 2] No such file or directory: '../csv_files/trade_dates_2023-05-27.csv'